<a href="https://colab.research.google.com/github/nightmarewarrior/homework/blob/main/%D0%AF%D0%B7%D1%8B%D0%BA%D0%BE%D0%B2%D0%B0%D1%8F_%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание № 4. Языковые модели

In [1]:
!pip install razdel

from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from IPython.display import Image
from IPython.core.display import HTML
from scipy.sparse import lil_matrix, csr_matrix, csc_matrix
from collections import Counter
import nltk
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

## Задание 1 (8 баллов).

В семинаре для генерации мы использовали предположение маркова и считали, что слово зависит только от 1 предыдущего слова. Но ничто нам не мешает попробовать увеличить размер окна и учитывать два или даже три прошлых слова. Для них мы еще сможем собрать достаточно статистик и, логично предположить, что качество сгенерированного текста должно вырасти.

Попробуйте сделать языковую модель, которая будет учитывать два предыдущих слова при генерации текста.
Сгенерируйте несколько текстов (3-5) и расчитайте перплексию получившейся модели.
Можно использовать данные из семинара или любые другие (можно брать только часть текста, если считается слишком долго). Перплексию рассчитывайте на 10-50 отложенных предложениях (они не должны использоваться при сборе статистик).


Подсказки:  
    - нужно будет добавить еще один тэг \<start>  
    - можете использовать тот же подход с матрицей вероятностей, но по строкам хронить биграмы, а по колонкам униграммы
    - тексты должны быть очень похожи на нормальные (если у вас получается рандомная каша, вы что-то делаете не так)
    - у вас будут словари с индексами биграммов и униграммов, не перепутайте их при переводе индекса в слово - словарь биграммов будет больше словаря униграммов и все индексы из униграммного словаря будут формально подходить для словаря биграммов (не будет ошибки при id2bigram[unigram_id]), но маппинг при этом будет совершенно неправильным

In [34]:
with open('2ch_corpus.txt', encoding='utf-8') as f:
    dvach = f.read()

In [35]:
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text

#norm_dvach = normalize(dvach)


In [38]:
from nltk.tokenize import sent_tokenize
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

sentences_dvach = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(dvach[:5000000])]

unigrams_dvach = Counter()
bigrams_dvach = Counter()
trigrams_dvach = Counter()

for sentence in sentences_dvach:
    unigrams_dvach.update(sentence)
    #bigrams_dvach.update(ngrammer(sentence, n=2))
    bigrams_dvach.update(ngrammer(sentence))
    trigrams_dvach.update(ngrammer(sentence, n=3))

matrix_dvach = lil_matrix((len(bigrams_dvach), len(unigrams_dvach)))

id2word_dvach = list(unigrams_dvach)
word2id_dvach = {word:i for i, word in enumerate(id2word_dvach)}

id2bigrams_dvach = list(bigrams_dvach)
word2idbigrams_dvach = {word:i for i, word in enumerate(id2bigrams_dvach)}

for ngram in trigrams_dvach:
    bigram, unigram = ngram.rsplit(maxsplit=1)
    #if word_bigram in word2idbigrams_dvach and word_unigram in word2id_dvach:
    matrix_dvach[word2idbigrams_dvach[bigram], word2id_dvach[unigram]] = (
            trigrams_dvach[ngram] / bigrams_dvach[bigram]
        )
matrix_dvach = csc_matrix(matrix_dvach)

In [39]:
def apply_temperature(probas, temperature):
    log_probas = np.log(np.maximum(probas, 1e-10))
    adjusted_log_probas = log_probas / temperature
    exp_probas = np.exp(adjusted_log_probas)
    adjusted_probabilities = exp_probas / np.sum(exp_probas)
    return adjusted_probabilities

def generate_temp(matrix, id2word, id2bigrams, word2id, word2idbigrams, n=100, start='<start> <start>', temperature=1.):
    text = start.split()
    current_idx = word2idbigrams[start]

    for i in range(n):
        chosen = np.random.choice(matrix.shape[1], p=apply_temperature(matrix[current_idx].toarray()[0], temperature=temperature))
        text.append(id2word[chosen])

        if id2word[chosen] == '<end>':
            current_idx = word2idbigrams['<start> <start>']
            text.extend(['<start>', '<start>'])
        else:
            current_idx = word2idbigrams[' '.join(text[-2:])]
    return ' '.join(text)

In [7]:
print(generate_temp(matrix_dvach, id2word_dvach, word2id_dvach, id2bigrams_dvach, word2idbigrams_dvach,  n=40, temperature=0.09).replace('<start> <start>', '').replace('<end>', '\n'))

 а я вот например автократ от милитаристов может построить крупный корабль у тебя есть полезные проекты на крестах пхп или тому подобном говне совковые традиции где нужно сидеть строго 8 часов в день 
  а если я не знаю как


In [ ]:
print(generate_temp(matrix_dvach, id2word_dvach, word2id_dvach, id2bigrams_dvach, word2idbigrams_dvach,  n=40, temperature=1.0).replace('<start> <start>', '').replace('<end>', '\n'))

 и динамичным главным героем че бля самый смешной здесь 
  согласно исландским сагам харальд прекрасноволосый имел много жен и детей давай стругать 
  проиграл со внезапной метафоры на карьерную лестницу


In [ ]:
print(generate_temp(matrix_dvach, id2word_dvach, word2id_dvach, id2bigrams_dvach, word2idbigrams_dvach,  n=40, temperature=1.4).replace('<start> <start>', '').replace('<end>', '\n'))

 get out of her life 
  сайдомв смысле спиноффом 
  но мамку зато видел 
  поэтому думаю вписать еще одну годную свежую метроидванию вспомнил визульно криповатая как метроиды или какая-нибудь contra но если уж вы все знаете 
  ук ебут


Почему-то выше 1.4 температура не повышается - выдает KeyError.. И как её исправить я не знаю..

In [ ]:
print(len(word2id_dvach) < len(word2idbigrams_dvach))

True


In [9]:
def perplexity(logp, N):
    return np.exp((-1/N) * logp)

def compute_join_proba_markov_assumption(text, matrix, word2id, word2idbigrams, trigrams):
    tokens = ['<start>'] + ['<start>'] + normalize(text) + ['<end>']
    trigrams_tokens = ngrammer(tokens, n=3)
    log_prob = 0
    for trigram in trigrams_tokens:
        bigram, unigram = trigram.rsplit(maxsplit=1)
        if bigram in word2idbigrams and unigram in word2id:
            bigram_idx = word2idbigrams[bigram]
            unigram_idx = word2id[unigram]
            prob = matrix[bigram_idx, unigram_idx]
            if prob > 0:
                log_prob += np.log(prob)
            else:
                log_prob += np.log(2e-5)
        else:
            log_prob += np.log(2e-5)
    return log_prob, len(tokens) - 3


corpus = dvach[5000000:]
norm = normalize(corpus)
vocab = Counter(norm)
probas = Counter({word:c/len(norm) for word, c in vocab.items()})

In [ ]:
text = (generate_temp(matrix_dvach, id2word_dvach, word2id_dvach, id2bigrams_dvach, word2idbigrams_dvach).replace('<start> <start>', '').replace('<end>', '\n'))

In [ ]:
ps = []
for sent in sent_tokenize(dvach[:5000000]):
  prob, N = compute_join_proba_markov_assumption(
        sent, matrix_dvach, word2id_dvach, word2idbigrams_dvach, trigrams_dvach
        )
  if not N:
        continue
  ps.append(perplexity(prob, N))


In [ ]:
np.mean(ps)

1448.3196692714876

## Задание № 2* (2 балла).

Измените функцию generate_with_beam_search так, чтобы она работала с моделью, которая учитывает два предыдущих слова.
Сравните получаемый результат с первым заданием.
Также попробуйте начинать генерацию не с нуля (подавая \<start> \<start>), а с какого-то промпта. Но помните, что учитываться будут только два последних слова, так что не делайте длинные промпты.

In [40]:
class Beam:
    def __init__(self, sequence: list, score: float):
        self.sequence: list = sequence
        self.score: float = score

In [52]:
def generate_with_beam_search(matrix, id2word, id2bigrams, word2id, word2idbigrams, n=100, max_beams=5, start='<start> <start>'):
    initial_node = Beam(sequence=start.split(), score=np.log1p(0))
    beams = [initial_node]

    for i in range(n):
        new_beams = []
        for beam in beams:
            if beam.sequence[-1] == '<end>':
                new_beams.append(beam)
                continue
            last_bigram = ' '.join(beam.sequence[-2:])
            if last_bigram not in word2idbigrams:
                continue
            last_id = word2idbigrams[last_bigram]
            probas = matrix[last_id].toarray()[0]
            top_idxs = probas.argsort()[:-(max_beams+1):-1]
            for top_id in top_idxs:
                if not probas[top_id]:
                    break
                new_sequence = beam.sequence + [id2word[top_id]]
                new_score = (beam.score + np.log1p(probas[top_id])) / len(new_sequence)
                new_beam = Beam(sequence=new_sequence, score=new_score)
                new_beams.append(new_beam)
        beams = sorted(new_beams, key=lambda x: x.score, reverse=True)[:max_beams]
        if all(beam.sequence[-1] == '<end>' for beam in beams):
            break
    sorted_sequences = sorted(beams, key=lambda x: x.score, reverse=True)
    sorted_sequences = [" ".join(beam.sequence) for beam in sorted_sequences]
    return sorted_sequences

In [53]:
result = generate_with_beam_search(matrix_dvach, id2word_dvach, word2id_dvach, id2bigrams_dvach, word2idbigrams_dvach)


In [54]:
result

['<start> <start> я не знаю что это за хуйня <end>',
 '<start> <start> я не знаю <end>',
 '<start> <start> ну и что <end>',
 '<start> <start> я не знаю что это не так <end>',
 '<start> <start> я не знаю как это будет называться <end>']

In [55]:
generate_with_beam_search(matrix_dvach, id2word_dvach, word2id_dvach, id2bigrams_dvach, word2idbigrams_dvach, start='<start> анимублядский')

['<start> анимублядский webm-треддля приличных анимублядей и прочих аутистов <end>',
 '<start> анимублядский webm-треддля приличных анимублядей и прочих лулзов <end>',
 '<start> анимублядский webm-треддля приличных анимублядей и прочих прототипов этахудожник <end>',
 '<start> анимублядский webm-треддля приличных анимублядей и прочих прототипов остальные повторяют <end>',
 '<start> анимублядский webm-треддля приличных анимублядей и прочих подобных по сложности данжей <end>']

In [56]:
generate_with_beam_search(matrix_dvach, id2word_dvach, word2id_dvach, id2bigrams_dvach, word2idbigrams_dvach, start='<start> <start> тред не читал')

['<start> <start> тред не читал почитай русскую классику типа достоевского чехова пушкина <end>',
 '<start> <start> тред не читал почитай русскую классику <end>',
 '<start> <start> тред не читал <end>',
 '<start> <start> тред не читал джойса но полагаю все это время в тюрьме накопится желание калечить и убивать уже никого небыло онлайн громили неписей <end>',
 '<start> <start> тред не читал джойса но полагаю все это время в тюрьме накопится желание калечить и убивать уже никого не ебал педоборец так как они вообще не понимаю <end>']